# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

city_df = pd.read_csv("../weatherpy/city_weather.csv")
city_df

,Unnamed: 0,city,latitude,longitude,date,temperature,humidity,cloudiness,wind speed
0,0,Rikitea,-23.1203,-134.9692,1643609638,77.27,63,100,5.53
1,1,Dikson,73.5069,80.5464,1643609639,-3.37,94,100,22.30
2,2,Chicama,-7.8447,-79.1469,1643609639,64.94,85,96,3.36
3,3,Ushuaia,-54.8000,-68.3000,1643609639,42.46,75,100,8.05
4,4,Butaritari,3.0707,172.7902,1643609530,82.06,77,55,18.97
...,...,...,...,...,...,...,...,...,...
530,530,Tazmalt,36.3868,4.3989,1643609839,45.97,37,0,3.51
531,531,Turukhansk,65.8167,87.9833,1643609839,1.45,93,99,12.35
532,532,Flinders,-34.5833,150.8552,1643609840,82.74,71,28,4.00
533,533,Nouadhibou,20.9310,-17.0347,1643609840,64.38,72,100,19.57


In [9]:
g_key

'YAIzaSyB9bQ8N_A9baEchLGipW3m-7cYY6SXu168'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [12]:
locations = city_df[['latitude','longitude']]
rating=city_df['humidity']

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
